In [28]:
!!auto-py-to-exe

^C


[]

In [19]:
import pandas as pd
gpa = pd.read_csv('sfrbx_gps_alm.csv', index_col=0)
gpe = pd.read_csv('sfrbx_gps_eph.csv', index_col=0)
gla = pd.read_csv('sfrbx_glonass_alm.csv', index_col=0)
gle = pd.read_csv('sfrbx_glonass_eph.csv', index_col=0)
with open('sfrbx_gps_data.txt', 'r') as file:
    gp_data = eval(file.read())

In [25]:
from math import pi, floor, sin, cos, atan, sqrt, tan
import Constants

SV = 4
alm = gla.loc[SV-1]
eph = gle.loc[SV-1]
# print(alm)
print(f'SV: {SV} glonass')

def get_glo_year(N4, N):
    if 1 <= N <= 366:
        J=1
    elif 367 <= N <= 731:
        J = 2
    elif 732 <= N <= 1096:
        J = 3
    elif 1097<= N <= 1461:
        J = 4
    else:
        J = 0 # TODO - error
    return 1996 + 4 * (N4-1) + (J-1)

def get_glo_dNA(N4, NA, N):
    k = 1461 if N4 != 27 else 1460
    return N - NA - round((N-NA)/k)*k

#TODO: переименовать N в NA в преобраовании ГЛОНАСС 
t1 = 81300 # 22:35
print(f'{t1//3600}h : {(t1%3600//60)}m')
N = 204 # 22.07.2024
# t1, N - время и день, на которые проводится расчет

GM = 398600441.8e6
ae = 6378136
J20 = 1082.62575e-6
wz = Constants.OmegaEarthDot

NA = eph.N
N4 = eph.N4
t_lambda = alm.t_lambda_n
dT = alm.delta_T_n
di = alm.delta_i_n
dT_dot = alm.delta_T_dot_n
e = alm.eps_n
w_A = alm.omega_n
lambda_A = alm.lambda_n

# для проверки
# N = 1453
# t1 = 51300
# NA = 1452
# t_lambda = 33571.625
# dT = 0.01953124999975
# dT_dot= 6.103515625E-05
# lambda_A = -0.293967247009277 * pi
# w_A = 0.57867431640625 * pi
# e = 0.000432968139648438
# di = -0.00012947082519531 * pi

# 1 - интервал прогноза dt_pr в сек
dNa = get_glo_dNA(N4=N4, NA=NA, N=N)
dt_pr = dNa * 86400 + (t1 - t_lambda)
print(1, dt_pr)

# 2 - количество целых витков W на интервале прогноза
Tdr_avg = 43200
W = floor(dt_pr / (Tdr_avg + dT))
print(2, W)

# 3 - текущее наклонение
i_avg = 63 / 180 * pi
i = di + i_avg
print(3, i)
#TODO: результат не сходится с примеров в документации
# #i = 1.1305666106990377

# 4 - средний драконический период на витке W+1 и среднее движение
Tdr = Tdr_avg + dT + (2*W + 1) * dT_dot
n = 2*pi / Tdr
print(4, Tdr, n)

# 5 - рассчитывается большая полуось последовательными приближениями (m=0, 1, 2...)
T1 = Tdr
a0 = -1
a1 = 0
p1 = 0
while abs(a0-a1) > 1e-3:
    a1 = ((T1/2/pi)**2 * GM) ** (1.0/3.0)
    p1 = a1 * (1-e**2)
    Ttt = (2-2.5*sin(i)**2) * ((1-e**2) ** 1.5 / (1 + e * cos(w_A * pi)) ** 2) + (1 + e * cos(w_A * pi)) ** 3 / (1 - e ** 2)
    Tdown = 1 - 1.5 * J20 * (ae/p1)** 2 * Ttt
    T1 = Tdr / Tdown
    a0 = a1
# T1, a1, p1
print(5, a1, p1, T1)

# 6 - текущее значение долготы восходящего узла и аргумента перигея
lmb = lambda_A - (wz + 1.5*J20*n*(ae/p1)**2*cos(i))* dt_pr
w = w_A - 0.75 * J20 * n * (ae/p1)**2 * (1-5*cos(i)**2) * dt_pr
print(6, lmb, w)

# 7 - среднее значение долготы на момент прохождения восходящего узла
E0 = -2 * atan(sqrt((1-e)/(1+e)) * tan(w/2))
L1 = w + E0 - e*sin(E0)
print(7, E0, L1)

# 8 - текущее значение долготы НКА
L = L1 + n * (dt_pr - (Tdr_avg + dT)*W - dT_dot*W**2)
L2 = L
print(8, L)

# 9 - для повышения точности, можно опустить
# TODO: добавить п.9

# 10 - эксцентрическая аномалия
E1 = 10 + E0
while abs(E1-E0) > 1e-9:
    E0 = E1
    E1 = L - w + e * sin(E0)
E = E1
# 11 истинняа аномалия и аргумент широты
nu = 2 * atan( sqrt((1+e)/(1-e)) * tan(E/2))
u = nu + w

# 12 - координаты ЦМ в прямоугольной ск
p = a1 * (1-e**2)
r = p / (1+e * cos(nu))
X = r * (cos(lmb)*cos(u)-sin(lmb)*sin(u)*cos(i))
Y = r * (sin(lmb)*cos(u)+cos(lmb)*sin(u)*cos(i))
Z = r * sin(u) * sin(i)

print(X, Y, Z)
import Transformations
print(*Transformations.ecef2lla(X, Y, Z))





SV: 4 glonass
22h : 35m
1 65857.65625
2 1
3 1.1384282622369515
4 40544.14013671875 0.00015497147765354202
5 25508097.942972608 25508074.08936014 40548.01622119976
6 -2.4051514765702624 -1.8205149636015134
7 1.8195778209248723 -0.001874396771556872
8 3.920998751249718
8408345.595348286 17744374.917798135 -16248767.427835636
-39.65532539186282 64.64560868763841 19117523.17765918


In [18]:
n * ( dt_pr - Tdr_avg * W)

2.5784864391716438